# Introdução

Por meio do [*site* da Defesa Civil](http://www.defesacivil.sp.gov.br/) (mais específicamente nesse [*link*](http://www.defesacivil.sp.gov.br/coordenadores-regionais-de-defesa-civil/)) foi possível obter diversas informações sobre os Coordenadores que atuam nas 'Coordenadorias Regionais de Proteção e Defesa Civil - REPDEC'.

In [ ]:
import os
import re
import time
import random
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

from bs4 import BeautifulSoup
from osgeo import gdal, osr
from tqdm.notebook import trange, tqdm

<br>

# Dados Tabulares

## Scrapy *Site*

Iniciamente foi obtido o código fonte da paǵina, usando o *BeautifulSoup*.

In [ ]:
url = 'http://www.defesacivil.sp.gov.br/coordenadores-regionais-de-defesa-civil/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
tag = soup.find('div', {'id': 'content'}).text

<br>

## Ajuste da tabela

A partir disso, montei uma lista com todos os valores que estavam na página e converti a lista em uma tabela. Adicionei uma coluna contendo o valor do índice quando, na linha da tabela, houvesse a palavra "REPDEC e preenchi essa tabela.

In [ ]:
list_main = []
for i in str(tag).split('\n'):
    if i == '':
        pass
    else:
        list_main.append(i)

# Create Table
df = pd.DataFrame({'data': list_main})
df

In [ ]:
# Define os marcos iniciais
df['interval'] = df.apply(lambda x: x.name if 'REPDEC' in x['data'] else np.nan, axis=1)

# Result
df.head(10)

In [ ]:
# Remove Espaços em Branco
df['data'] = df.apply(lambda x: np.nan if x['data'] == '\xa0' else x, axis=1)

# Result
df.head(10)

In [ ]:
# Preenche Coluna
df['interval']  = df['interval'].ffill()

# Result
df.head(10)

In [ ]:
# Elimina Linhas com NaN
df = df.dropna()

# Result
df.head(10)

<br>

Criei sub-listas, de tamanhos diferentes, para cada uma desses conjuntos, por meio desse índice e converti em uma tabela.

In [ ]:
list_temps = []
for i in list(set(df['interval'])):
    df_temp = df[df['interval'] == i]
    list_temp = list(df_temp['data'])
    list_temps.append(list_temp)

df = pd.DataFrame(
    list_temps,
    columns = [
        'repdec',
        'coord_regional_nome',
        'coord_regional_mail',
        'coord_regional_tel',
        'coord_adj1_nome',
        'coord_adj1_mail',
        'coord_adj1_tel',
        'coord_adj2_nome',
        'coord_adj2_mail',
        'coord_adj2_tel',
        'coord_adj3_nome',
        'coord_adj3_mail',
        'coord_adj3_tel',
        'coord_adj4_nome',
        'coord_adj4_mail',
        'coord_adj4_tel',
        'coord_adj5_nome',
        'coord_adj5_mail',
        'coord_adj5_tel',
        'coord_adj6_nome',
        'coord_adj6_mail',
        'coord_adj6_tel',
        'coord_adj7_nome',
        'coord_adj7_mail',
        'coord_adj7_tel',
    ])

df.head()

<br>

Com a tabela bruta, passei a limpa-la, "splitando" os campos pelo caractere ":", além de outras correções.

In [ ]:
# Exceção: em uma célula há duas vezes o ":"
df['coord_adj1_tel'] = df['coord_adj1_tel'].astype(str).apply(lambda x: x.replace('Ramal:', 'Ramal '))

# Remove tudo que vem antes do :
df = df.applymap(lambda x: x.split(':')[-1], na_action='ignore')

# Divide Células em Duas Colunas
df[['repdec_nome','repdec_cod']] = df['repdec'].str.split('–', expand=True)
df[['del','repdec_cod']] = df['repdec'].str.split('/', expand=True)
df.drop(['del', 'repdec'], axis=1, inplace=True, errors='ignore')

# Results
df.head()

## Funções

Funções para renomear campos

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  '  : ' ',
        
        # Basics
        ' Com '  : ' com ',
        ' Sobre ': ' sobre ',
        ' Da '   : ' da ',
        ' De '   : ' de ',
        ' Do '   : ' do ',
        ' Das '  : ' das ',
        ' Dos '  : ' dos ',
        ' A '    : ' a ',
        ' As '   : ' as ',
        ' Ao '   : ' ao ',
        ' Aos '  : ' aos ',
        ' E '    : ' e ',
        ' O '    : ' o ',
        ' Os '   : ' os ',
        
        # Erros
        '1ºten'  : '1º Ten',
        
        # Abreviações
        'Subten ' : 'Subtenente ',
        'Sub Ten' : 'Subtenente ',
        'Cap '    : 'Capitão ',
        'Ten '    : 'Tenente ',
        'Maj '    : 'Major ',
        'Cel '    : 'Coronel ',        
        'Sgt '    : 'Sargento ',
        ' Pm '    : ' PM ',

        # Empty
        'None' : '',
        'none' : '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
def rename_tel(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  '  : ' ',

        # Basics
        ' – ' : '-',
        ')'   : ') ',
        ' ('  : ' / (',
        ' / ' : '/',
        '//'  : '/',
        '/'   : ' / ',
        '  '  : ' ',
        
        # Empty
        'None' : '',
        'none' : '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
def rename_mail(x):
    x = x.lower()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  '  : ' ',
        
        # Basic
        ' '   : '/',
        ';'   : '/',
        ','   : '/',
        '//'  : '/',
        '/'   : ' / ',
        '  '  : ' ',
        
        # Empty
        'None' : '',
        'none' : '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

<br>

## Renomeando Campos em Colunas

In [ ]:
# Definindo lista de colunas que serão empregadas funções.
list_nome = []
list_tel  = []
list_mail = []

for col in list(df.columns):
    if 'nome' in col:
        list_nome.append(col)
    if 'tel' in col:
        list_tel.append(col)
    if 'mail' in col:
        list_mail.append(col)

# Adicionando manualmente outras colunas.
list_nome.append('repdec_nome')
list_nome.append('repdec_cod')

# Aplicando funções
for field in list_nome:
    df[field] = df[field].astype(str).apply(lambda x: rename_nome(x))
for field in list_tel:    
    df[field] = df[field].astype(str).apply(lambda x: rename_tel(x))
for field in list_mail:    
    df[field] = df[field].astype(str).apply(lambda x: rename_mail(x))

<br>

E, por fim, reordenando colunas.

In [ ]:
# Reordena Colunas
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df.reindex(cols, axis=1)

# Resultados
df.head()

## *Splita* Telefones

In [ ]:
# Definindo lista de colunas que serão empregadas funções.
list_tel  = []

for col in list(df.columns):
    if 'tel' in col:
        list_tel.append(col)

# Aplicando funções
for field in list_tel:
    tel_01      = '{}01'.format(field)
    tel_02      = '{}02'.format(field)
    tel_01ramal = '{}01ramal'.format(field)
    tel_02ramal = '{}02ramal'.format(field)
    
    # Split Telefone
    try:    df[tel_01]      = df[field].str.split(' / ', n=1, expand=True)[0]
    except: df[tel_01]      = None
    try:    df[tel_01ramal] = df[tel_01].str.split(' Ramal ', n=1, expand=True)[1]
    except: df[tel_01ramal] = None
    try:    df[tel_01]      = df[tel_01].str.split(' Ramal ', n=1, expand=True)[0]
    except: df[tel_01]      = None
    
    
    try:    df[tel_02]      = df[field].str.split(' / ', n=1, expand=True)[1]
    except: df[tel_02]      = None
    try:    df[tel_02ramal] = df[tel_02].str.split(' Ramal ', n=1, expand=True)[1]
    except: df[tel_02ramal] = None
    try:    df[tel_02]      = df[tel_02].str.split(' Ramal ', n=1, expand=True)[0]
    except: df[tel_02]      = None
    
    # Deleta Coluna
    df.drop(field, axis=1, inplace=True, errors='ignore')

## *Splita* E-mails

In [ ]:
list_mail = []

for col in list(df.columns):
    if 'mail' in col:
        list_mail.append(col)

# Aplicando funções
for field in list_mail:
    mail_01      = '{}01'.format(field)
    mail_02      = '{}02'.format(field)
    
    # Split Telefone
    try:    df[mail_01]      = df[field].str.split(' / ', n=1, expand=True)[0]
    except: df[mail_01]      = None
    try:    df[mail_02]      = df[field].str.split(' / ', n=1, expand=True)[1]
    except: df[mail_02]      = None

    # Deleta Coluna
    df.drop(field, axis=1, inplace=True, errors='ignore')

## Reordena Colunas

In [ ]:
# Reordena Colunas
cols = df.columns.tolist()

cols.sort()
cols = cols[-2:] + cols[-9:-2] + cols[:-9]

df = df.reindex(cols, axis=1)
cols

## Salva

In [ ]:
# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_defesacivil.csv'),
    index=False,
)
df